In [3]:
import lightgbm as lgb
from init_ulits import *
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import math

In [4]:
train_data = load_data('tap_fun_train.csv')
test_data = load_data('tap_fun_test.csv')

## 数据分割成4部分
1. 训练集 train
2. 验证集 eval
3. 测试集 test
4. 预测集 predict

In [5]:
Y = train_data['prediction_pay_price']
X = train_data.drop(['prediction_pay_price','user_id','register_time'], axis=1)
X_predict = test_data.drop(['user_id','register_time'], axis=1)

In [6]:
X, X_test, Y, Y_test = train_test_split(X, Y, random_state = 33, test_size = 0.1)
X_train, X_eval, Y_train, Y_eval = train_test_split(X, Y, random_state = 33, test_size = 0.1)

In [7]:
lgb_train = lgb.Dataset(X_train, Y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_eval, Y_eval, reference=lgb_train,free_raw_data=False)

In [174]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mean_squared_error',
    'learning_rate': 0.01,
    'num_leaves':127,
    'max_bin':800,
    'min_data_in_leaf':2,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.7,
    'bagging_freq':1,
    'lambda_l1': 0.5,
    'lambda_l2': 0,
    'min_split_gain': 0,
    'max_depth':7
 }
eresult = {}

In [175]:
gbm = lgb.train(params,
                lgb_train,num_boost_round=4000, 
                valid_sets=[lgb_eval,lgb_train],
                valid_names = ['eval', 'train'],
                verbose_eval=10,
                evals_result=eresult,
                early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds.
[10]	train's l2: 6351.29	eval's l2: 7489.56
[20]	train's l2: 5540.66	eval's l2: 6894.57
[30]	train's l2: 4870.9	eval's l2: 6475.13
[40]	train's l2: 4298.55	eval's l2: 6092.22
[50]	train's l2: 3813.6	eval's l2: 5802.7
[60]	train's l2: 3394.84	eval's l2: 5565.92
[70]	train's l2: 3046.87	eval's l2: 5351.1
[80]	train's l2: 2734.01	eval's l2: 5181.42
[90]	train's l2: 2475.89	eval's l2: 5075.19
[100]	train's l2: 2259.26	eval's l2: 4972.31
[110]	train's l2: 2076.59	eval's l2: 4914.53
[120]	train's l2: 1912.14	eval's l2: 4859.17
[130]	train's l2: 1771.06	eval's l2: 4849.88
[140]	train's l2: 1647.69	eval's l2: 4815.58
[150]	train's l2: 1545.56	eval's l2: 4801.36
[160]	train's l2: 1453.65	eval's l2: 4779.88
[170]	train's l2: 1374.26	eval's l2: 4781.92
[180]	train's l2: 1301.45	eval's l2: 4747.34
[190]	train's l2: 1240.02	eval's l2: 4730.85
[200]	train's l2: 1181.39	eval's l2: 4720.42
[210]	train's l2: 1133.26	eval's l2: 4711.92
[220]

### 测试集cost

In [176]:
predict =  gbm.predict(X_test, num_iteration=gbm.best_iteration)
cost = metrics.mean_squared_error(Y_test, predict)
sqlrtcost = math.sqrt( cost )
print(sqlrtcost)

82.0045763406069


### 验证集cost

In [177]:
predict =  gbm.predict(X_eval, num_iteration=gbm.best_iteration)
cost = metrics.mean_squared_error(Y_eval, predict)
sqlrtcost = math.sqrt( cost )
print(sqlrtcost)

68.62117001113482


In [22]:
# predict[ predict < 0.5 ] = 0
# predict[ (0.5 < predict) & (predict < 1.5) ] = 0.99
# predict[ (1.5 < predict) &  (predict < 2.5) ] = 1.98
# predict[ (2.5 < predict) &  (predict < 3.5) ] = 2.97
# print(predict)

[ 0.  0.  0. ...,  0.  0.  0.]


## 预测并输出

In [178]:
Y_predict =  gbm.predict(X_predict, num_iteration=gbm.best_iteration)
# Y_predict[ Y_predict < 0.5 ] = 0
# Y_predict[ (0.5 < Y_predict) & (Y_predict < 1.5) ] = 0.99
# Y_predict[ (1.5 < Y_predict) &  (Y_predict < 2.5) ] = 1.98
# Y_predict[ (2.5 < Y_predict) &  (Y_predict < 3.5) ] = 2.97

In [179]:
df = test_data[['user_id']]
df.insert(1,'prediction_pay_price',Y_predict)
# df['prediction_pay_price'] = Y_predict

# df['prediction_pay_price'].round(2)
df.to_csv("20180708.csv",index=None,sep=',')